In [ ]:
import os
import pandas as pd
from tqdm import tqdm
import time
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 🔑 Set your API keys
os.environ["OPENAI_API_KEY"] = ""   
os.environ["PINECONE_API_KEY"] = ""

# --- Config ---
INDEX_NAME = "curewise-medical-rag"
EMBED_MODEL = "text-embedding-3-small"
BATCH_SIZE = 50  # smaller batch to avoid issues
CHUNK_SIZE = 4000  # each chunk safely under 40 KB after JSON encoding
CHUNK_OVERLAP = 200

# --- Init OpenAI and Pinecone ---
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

# Create index if not exists
if INDEX_NAME not in pc.list_indexes().names():
    pc.create_index(
        name=INDEX_NAME,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    print("⏳ Waiting for index to be ready...")
    while True:
        desc = pc.describe_index(INDEX_NAME)
        if desc.status["ready"]:
            break
        time.sleep(5)
    print("✅ Index created!")

# Connect to index
index = pc.Index(INDEX_NAME)

# --- Load CSV ---
df = pd.read_csv("/Users/tkarim45/Documents/Personal Github Repositories/CureWise-AI-Medical-Healthcare/backend/data/rag/data.csv")
assert all(c in df.columns for c in ["qtype", "Question", "Answer"])

# --- Embed text ---
def embed_texts(texts):
    response = client.embeddings.create(model=EMBED_MODEL, input=texts)
    return [d.embedding for d in response.data]

# --- Initialize text splitter ---
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP
)

# --- Upsert in batches with chunking ---
for i in tqdm(range(0, len(df), BATCH_SIZE), desc="Uploading"):
    batch = df.iloc[i:i+BATCH_SIZE]
    vectors = []

    for idx, row in batch.iterrows():
        combined = f"Q: {row['Question']}\nA: {row['Answer']}"
        chunks = text_splitter.split_text(combined)
        embeddings = embed_texts(chunks)

        for j, (chunk, emb) in enumerate(zip(chunks, embeddings)):
            metadata = {
                "qtype": row["qtype"],
                "text": chunk
            }
            vectors.append((f"{idx}_{j}", emb, metadata))

    if vectors:
        index.upsert(vectors=vectors)

    stats = index.describe_index_stats()
    print(f"Uploaded so far: {stats['total_vector_count']} vectors")

# Final stats
stats = index.describe_index_stats()
print("🎉 Upload complete!")
print(f"Total vectors in index: {stats['total_vector_count']}")


⏳ Waiting for index to be ready...
✅ Index created!


Uploading:   0%|          | 1/329 [00:26<2:26:09, 26.74s/it]

Uploaded so far: 0 vectors


Uploading:   1%|          | 2/329 [00:54<2:29:03, 27.35s/it]

Uploaded so far: 104 vectors


...

Uploading: 100%|█████████▉| 328/329 [2:48:00<00:24, 24.35s/it]

Uploaded so far: 17706 vectors


Uploading: 100%|██████████| 329/329 [2:48:04<00:00, 30.65s/it]

Uploaded so far: 17713 vectors


🎉 Upload complete!
Total vectors in index: 17713


In [ ]:
query = "What is acne?"

# Embed query
query_emb = client.embeddings.create(
    model="text-embedding-3-small",
    input=query
).data[0].embedding

# Search Pinecone
results = index.query(vector=query_emb, top_k=3, include_metadata=True)

for match in results.matches:
    print(f"Score: {match.score:.3f}")
    print("Q:", match.metadata["question"])
    print("A:", match.metadata["answer"])
    print("---")
